In [12]:
import pandas as pd
import numpy as np

from itertools import combinations
import math

I copy my code here from problem 124 to get the rads of each number. I additionally include the `math.gcd` function since it is written in C.

The nice property of rad here is that $\text{rad}(abc) = \text{rad}(\text{rad}(a) \cdot \text{rad}(b) \cdot \text{rad}(c))$. However, since $a,b,c$ are coprime by condition $1$, this means $\text{rad}(\text{rad}(a) \cdot \text{rad}(b) \cdot \text{rad}(c)) = \text{rad}(a) \cdot \text{rad}(b) \cdot \text{rad}(c)$. So the biggest rad we have to calculate is $119999$, which can be found pretty quickly.

If we quickly check for the $1000$ case, to confirm we are on the right path, we notice that particular $c$ may not be unique--in other words if two combinations $a_1, b_1, c$ and $a_2, b_2, c$ are both abc-hits, then we should count both of them. That is how we end up with thirty one adding up to $12523$. However, if we just change the number to $120000$ we quickly see that it runs way too slow, certainly too slow for under 1 minute.

#### Attempt to speed-up 1

To speed up the process we need to reduce the search space to only comprime integers. The first thing to notice is since $a + b = c$ and $gcd(a,b) = gcd(a+b, a) = gcd(a+b, b)$, we only need to calculate $gcd(a,b)$ each time. However this means we just need to find a fast way of generating coprime pairs $(a,b)$. From researching, there is an algorithm for generating coprime pairs on the Wikipedia page for [coprime integers][1]. We start with two "root" pairs $(a,b) = (1,2)$ and $(a,b) = (1,3)$ and we can generate all coprime integers using three branches:
1. $(b, 2b - a)$
2. $(b, 2b + a)$
3. $(a, b + 2a)$

However, even this method is too slow. From further research, we can also generate coprime pairs using concepts from Farey sequences (like problem 73). Finally, while scouring for the answer, I realized that the gcd algorithm is faster for smaller numbers and that $gcd(a,b) \neq 1 \iff gcd(rad(a), rad(b)) \neq 1$ since they are comprised of the same prime factors and that $rad(a) < a$ and $rad(b) < b$. So we can just check $gcd(rad(a), rad(b))$. Even this was to no avail.

#### After reading forum

After completing the problem (using a long 10 minute method) with very little luck on speeding up, I looked at the forum to find any improvements to be made, in particular to reducing the search space. What I found was that we can use the work from problem 124 on sorting radicals to improve the result. 
1. Since $a < b$ and $a + b = c$, we know that $a + b < c < 2b$--this tells us it's easiest to iterate on $b$ first since, we can set the bound on $c$ based on $b$ and then just calculate $a = c-b$.
2. Since $c < 2b$ and $rad(c) > 2 \implies \frac{1}{rad(c)} < \frac{1}{2}$, we have
$$
rad(abc) = rad(a) \cdot rad(b) \cdot rad(c) < c \implies rad(a) < \frac{c}{rad(b) \cdot rad(c)} < \frac{2b}{2 rad(b)} = \frac{b}{rad(b)}
$$
This tells us we can just look at certain values for $rad(a)$.
3. We can use the sorted list to cutoff only certain values for $rad(a)$.

Using this sorted list method with a cache speeds it up to about 8s, which is a huge improvement!

[1]: https://en.wikipedia.org/wiki/Coprime_integers#Generating_all_coprime_pairs
[2]: https://stackoverflow.com/questions/45699954/is-farey-sequence-the-fastest-way-to-generate-coprime-pairs

In [86]:
def sieve(n):
    arr = [0,0,1] + [1,0]*(n//2 + 1)
    i = 3
    while i*i <= n:
        if arr[i]:  
            arr[i*i::2*i] = [0]*len(arr[i*i::2*i])
        i += 2

    ret = []
    for (i, p) in enumerate(arr):
        if p:
            ret.append(i)

    return arr, ret

top_value = 120000
pbs, ps = sieve(top_value)

In [87]:
# rad sieve
rad = {i: 1 for i in range(1, top_value + 1)}
rad[1] = 1
for p in ps:
    for i in range(p, top_value + 1, p):
        rad[i] *= p

rad_sorted = sorted(rad.items(), key = lambda a: (a[1], a[0]))
max_val = rad_sorted[-1][1]

In [88]:
cntr = 0
s = 0
cutoff = 0
rad_sort_cache = {}

max_c = top_value
for b in range(2,max_c-1):
    rb = rad[b]
    cutoff_val = b // rb
    if cutoff_val not in rad_sort_cache:
        cutoff = 0
        while rad_sorted[cutoff][1] < cutoff_val:
            cutoff += 1

        rad_sort_cache[cutoff_val] = cutoff
    

    cutoff = rad_sort_cache[cutoff_val]    
    as_to_check = rad_sorted[:cutoff]
    for a, ra in as_to_check:
        if a >= b or a+b >= max_c:
            continue

        c = a+b
        rc = rad[c]

        if ra*rb*rc >= c or ra*rb >= b:
            continue
        
        if math.gcd(ra, rb) == 1:
            cntr += 1
            s += c
            print(cntr, a, b, c, s)


1 1 8 9 9
2 5 27 32 41
3 1 48 49 90
4 32 49 81 171
5 1 63 64 235
6 1 80 81 316
7 4 121 125 441
8 3 125 128 569
9 81 175 256 825
10 1 224 225 1050
11 1 242 243 1293
12 2 243 245 1538
13 7 243 250 1788
14 100 243 343 2131
15 13 243 256 2387
16 1 288 289 2676
17 32 343 375 3051
18 169 343 512 3563
19 5 507 512 4075
20 1 512 513 4588
21 27 512 539 5127
22 200 529 729 5856
23 81 544 625 6481
24 49 576 625 7106
25 1 624 625 7731
26 343 625 968 8699
27 104 625 729 9428
28 1 675 676 10104
29 25 704 729 10833
30 1 728 729 11562
31 640 729 1369 12931
32 1 960 961 13892
33 1 1024 1025 14917
34 5 1024 1029 15946
35 243 1088 1331 17277
36 1 1215 1216 18493
37 81 1250 1331 19824
38 8 1323 1331 21155
39 256 1331 1587 22742
40 1024 1377 2401 25143
41 81 1600 1681 26824
42 512 1675 2187 29011
43 243 1805 2048 31059
44 25 2023 2048 33107
45 23 2025 2048 35155
46 9 2048 2057 37212
47 625 2048 2673 39885
48 139 2048 2187 42072
49 81 2116 2197 44269
50 11 2176 2187 46456
51 1024 2187 3211 49667
52 2048 218

In [89]:
cntr, s

(456, 18407904)